# Simulating The Monty Hall Problem

The Monty Hall problem is a well-known puzzle in probability derived from an American game show, *Let’s Make a Deal*.

The original 1960s-era show was hosted by Monty Hall, giving this puzzle its name.

Intuition leads many people to get the puzzle wrong, and when the Monty Hall problem is presented in a newspaper or discussion list, it often leads to a lengthy argument in letters-to-the-editor and on message boards.

### How To Play

- The game show set has three doors. A prize such as a car or vacation is behind one door, and the other two doors hide a valueless prize called a Zonk; in most discussions of the problem, the Zonk is a goat.
- The contestant chooses one door. We’ll assume the contestant has no inside knowledge of which door holds the prize, so the contestant will just make a random choice.
- The smiling host Monty Hall opens one of the other doors, always choosing one that shows a goat, and always offers the contestant a chance to switch their choice to the remaining unopened door.
- The contestant either chooses to switch doors, or opts to stick with the first choice. Monty calls for the remaining two doors to open, and the contestant wins whatever is behind their chosen door.

Let’s say a hypothetical contestant chooses door #2. Monty might then open door #1 and offer the chance to switch to door #3. The contestant switches to door #3, and then we see if the prize is behind #3.

The puzzle is: what is the best strategy for the contestant? Does switching increase the chance of winning the car, decrease it, or make no difference?

The best strategy is to make the switch. It’s possible to analyze the situation and figure this out, but instead we’ll tackle it by simulating thousands of games and measuring how often each strategy ends up winning.

# Approach

Simulating one run of the game is straightforward:

- We will write a `simulate()` function that uses Python’s random module to pick which door hides the prize, the contestant’s initial choice, and which doors Monty chooses to open. An input parameter controls whether the contestant chooses to switch, and `simulate()` will then return a Boolean telling whether the contestant’s final choice was the winning door.

- Part of the reason the problem fools so many people is that in the three-door case the probabilities involved are $1/3$ and $1/2$, and it’s easy to get confused about which probability is relevant. Considering the same game with many more doors makes reasoning about the problem much clearer, so we’ll make the number of doors a configurable parameter of the simulation script.


# Solution

In [ ]:
# Import packages
import random

In [ ]:

def simulate(num_doors: int, switch: bool, verbose: bool = False) -> bool:
    """
    Simulate one round of the Monty Hall game.

    Args:
        num_doors: Total number of doors in the game.
        switch: Whether the contestant switches their choice.
        verbose: If True, prints detailed steps of the simulation.

    Returns:
        bool: True if the contestant wins, False otherwise.
    """

    # Randomly assign the winning door and contestant's initial choice.
    winning_door = random.randint(0, num_doors - 1)
    choice = random.randint(0, num_doors - 1)

    if verbose:
        print(f"Prize is behind door {winning_door + 1}")
        print(f"Contestant chooses door {choice + 1}")

    # Host opens all but the contestant's and the winning door.
    closed_doors = [door for door in range(num_doors) if door not in {winning_door, choice}]
    door_to_remove = random.choice(closed_doors)

    closed_doors.remove(door_to_remove)
    if verbose:
        print(f"Host opens door {door_to_remove + 1}")

    # If the contestant switches, they pick the only remaining closed door.
    if switch:
        available_door = closed_doors[0]
        if verbose:
            print(f"Contestant switches from door {choice + 1} to {available_door + 1}")
        choice = available_door

    # Determine if the contestant won.
    won = choice == winning_door
    if verbose:
        print("Contestant WON\n" if won else "Contestant LOST\n")

    return won

def run_trials(trials: int, num_doors: int = 3, verbose: bool = False) -> None:
    """
    Run multiple simulations of the Monty Hall game.

    Args:
        trials: Number of trials to run for both switching and non-switching strategies.
        num_doors: Total number of doors in the game.
        verbose: If True, prints detailed output for each trial.
    """

    winning_non_switchers = sum(simulate(num_doors, switch=False, verbose=verbose) for _ in range(trials))
    winning_switchers = sum(simulate(num_doors, switch=True, verbose=verbose) for _ in range(trials))

    print(f"Switching won {winning_switchers:5} times out of {trials} "
          f"({(winning_switchers / trials * 100):.2f}% of the time)")
    print(f"Not switching won {winning_non_switchers:5} times out of {trials} "
          f"({(winning_non_switchers / trials * 100):.2f}% of the time)")

In [ ]:
# Parameters
trials = 10_000
doors = 3
verbose = False

# Run the trials
run_trials(trials, doors, verbose)

    Switching won  6617 times out of 10000 (66.17% of the time)
Not switching won  3296 times out of 10000 (32.96% of the time)


# Lessons Learned

This approach to answering a question, where we randomly generate many possible inputs, calculate the outcomes, and summarize the results, is called Monte Carlo simulation and has a long history, having been first developed in the 1940s by mathematicians working on the Manhattan Project to build an atomic bomb.

In the case of the Monty Hall problem, the simulation is straightforward to program and we can figure out an analytical result, so it’s easy to inspect the output and verify that the program is correct. Often, though, simulations are for attacking problems too complicated to be solved beforehand and then checking for correctness is much harder. The programmers will need to carefully validate their code by unit-testing the simulation’s internal functions and adding checks for internal correctness. monty-hall.py does this in a small way with an assert statement that will raise an exception if the number of closed doors is not equal to $2$, which would indicate some sort of failure in the `simulate()` function’s logic or input data. We could potentially add more such checks:

- Assert the winning door is not a legal door:
  - $0 <= $ `winning_door` $ < $ `num_doors`
- Assert contestant choice is not a legal door:
  - $0 <= $ `choice` $ < $ `num_doors`

In our simple simulation, these assertions are never going to fail, but perhaps we might make changes to `simulate()` that make the function incorrect, or perhaps someday a bug in Python’s random module will come to light.


# References

- http://en.wikipedia.org/wiki/Monty_Hall_problem
    Discusses the history of the problem and various approaches to solving it.

- http://library.lanl.gov/cgi-bin/getfile?00326867.pdf “Stan Ulam, John von Neumann, and the Monte Carlo Method” (1987), by Roger Eckhardt, is a discussion of the very first Monte Carlo simulation and some of the mathematical problems encountered while implementing it. This first simulation modelled neutrons diffusing through fissionable material in an effort to determine whether a chain reaction would occur. (The PDF also features a discussion of how random number generators work, written by Tony Warnock.)

- http://www.youtube.com/watch?v=0W978thuweY A condensed episode of an episode of the original game show, showing Monty Hall’s quick wit in action. Notice that the original game is more complicated than the Monty Hall puzzle as described above because Monty has many more actions available to him: he can offer the choice of an unknown prize or an unknown amount of cash, or suggest trading in what you’ve already won for a different unknown prize.
    
- https://fiftyexamples.readthedocs.io/en/latest/monty-hall.html Original post of discussion and code
